In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

## Using a loop to retrieve data from past seasons

In [2]:
standing_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [3]:
years = list(range(2022, 2017, -1))

all_matches = []

In [4]:
for year in years: 
    data = requests.get(standing_url)
    soup = BeautifulSoup(data.text)
    standing_table = soup.select("table.stats_table")[0]

    links = standing_table.find_all("a")
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"http://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standing_url = f"https://fbref.com/{previous_season}"


    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", "")

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match = "Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = soup.find_all("a")
        links = [l.get("href") for l in links]
        links = [l for l in links if l and "all_comps/shooting/" in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match = "Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = pd.merge(left = matches, right = shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "La Liga"]
        team_data["Season"] = year
        team_data["Team"] = team_name

        all_matches.append(team_data)
        time.sleep(5)

In [5]:
match_df = pd.concat(all_matches)
#match_df.columns
#match_df.to_csv("test.csv")

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2022-08-13,21:00,La Liga,Matchweek 1,Sat,Home,D,0,0,Rayo Vallecano,...,Match Report,NaN,21.0,5.0,17.0,1.0,0.0,0.0,2022,Barcelona
1,2022-08-21,22:00,La Liga,Matchweek 2,Sun,Away,W,4,1,Real Sociedad,...,Match Report,NaN,15.0,7.0,14.6,0.0,0.0,0.0,2022,Barcelona
2,2022-08-28,19:30,La Liga,Matchweek 3,Sun,Home,W,4,0,Valladolid,...,Match Report,NaN,24.0,9.0,14.4,1.0,0.0,0.0,2022,Barcelona
3,2022-09-03,21:00,La Liga,Matchweek 4,Sat,Away,W,3,0,Sevilla,...,Match Report,NaN,18.0,5.0,16.0,2.0,0.0,0.0,2022,Barcelona
5,2022-09-10,18:30,La Liga,Matchweek 5,Sat,Away,W,4,0,Cádiz,...,Match Report,NaN,16.0,8.0,14.9,0.0,0.0,0.0,2022,Barcelona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-25,19:30,La Liga,Matchweek 34,Thu,Away,L,0,5,Sevilla,...,Match Report,NaN,12.0,3.0,17.7,0.0,0.0,0.0,2018,RayoVallecano
36,2019-04-28,20:45,La Liga,Matchweek 35,Sun,Home,W,1,0,Real Madrid,...,Match Report,NaN,11.0,3.0,23.9,0.0,1.0,1.0,2018,RayoVallecano
37,2019-05-04,13:00,La Liga,Matchweek 36,Sat,Away,L,1,4,Levante,...,Match Report,NaN,20.0,3.0,18.4,2.0,0.0,0.0,2018,RayoVallecano
38,2019-05-12,18:30,La Liga,Matchweek 37,Sun,Home,L,1,2,Valladolid,...,Match Report,NaN,16.0,5.0,19.2,1.0,0.0,0.0,2018,RayoVallecano
